In [79]:
import requests 
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import tiktoken
import openai
from openai import OpenAI

In [ ]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
es_client = Elasticsearch('http://localhost:9200')
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "homework-questions"
es_client.indices.create(index = index_name, body = index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [46]:
q3_search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

In [65]:
q4_search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [82]:
def es_search(search_query, print_score):
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        if print_score: 
            print(hit['_score'])
        result_docs.append(hit['_source'])

    return result_docs

In [67]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()
    
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    context = ""
    
    for doc in search_results:
        context = context + context_template.format(question = doc['question'], text = doc['text']) + "\n\n"
        
        
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [77]:
def llm(prompt):
    client = OpenAI()
        
    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages = [{"role":"user", "content":prompt}]
    )

    return response.choices[0].message.content

In [68]:
query = 'How do I execute a command in a running docker container?'

In [84]:
q3 = es_search(q3_search_query, True)

84.050095
75.54128
72.08518
51.04628
49.938507


In [85]:
q4 = es_search(q4_search_query, False)
print(q4[2]['question'])

How do I copy files from a different folder into docker container’s working directory?


In [86]:
q5 = build_prompt(query, es_search(q4_search_query, False))
len(q5)

1462

In [90]:
encoding = tiktoken.encoding_for_model("gpt-4o")
q6 = len(encoding.encode(q5))
q6
#encoding.decode_single_token_bytes(63842)

322

In [91]:
q7 = llm(build_prompt(query, es_search(q4_search_query, False)))
print(q7)

To execute a command in a running Docker container, you can use the `docker exec` command. Here's how you can do it:

1. First, find the container ID of the running container by using the `docker ps` command.
2. Then, use the `docker exec` command to run a command in the container.

For example, to start a bash session in the container:

```sh
docker ps (find the container-id)
docker exec -it <container-id> bash
```

This will open an interactive bash shell inside the running container.


In [95]:
in_tokens = q6
out_tokens = len(encoding.encode(q7))

(in_tokens * 0.005 / 1000 + out_tokens * 0.015 / 1000) * 1000

3.32